In [1]:
import pandas as pd
import os, sys
file_dir = '/Users/imsoyoung/Soyoung/Ajou/Project/DigitalPhenotype/DigitalPhenotype_overall/Code/Util'
sys.path.append(os.path.dirname(os.path.abspath
                                (os.path.abspath(file_dir))))

import Util.Utility as util
import datetime

# App Data

## App Sample1 (A)
- uid
- date (YYYY-mm-dd)
- stime, etime (YYYY-mm-dd HH:MM:SS) 
- packageName
- total : total app usage time
- isInUsable : if user is using or not (foreground or background)
- packageFullName : we can find category of app with this (+ google play store crawling)

In [2]:
app_sample1 = pd.read_csv('../../Sample_Data/A/App.csv', encoding='utf-8')
print(app_sample1.shape)
app_sample1.head()


(35002, 9)


,_id,uid,date,stime,etime,packageName,total,isInUsable,packageFullName
0,1,otfKOMf7VUaB5lxUARO0HNusxPB2,2018-12-03,2018-12-03 00:11:26,NaN,캐시워크,0,False,com.cashwalk.cashwalk
1,2,otfKOMf7VUaB5lxUARO0HNusxPB2,2018-12-03,2018-12-03 00:11:37,NaN,캐시워크,0,False,com.cashwalk.cashwalk
2,3,otfKOMf7VUaB5lxUARO0HNusxPB2,2018-12-03,2018-12-03 00:11:59,2018-12-03 00:12:10,캐시워크,11,True,com.cashwalk.cashwalk
3,4,otfKOMf7VUaB5lxUARO0HNusxPB2,2018-12-03,2018-12-03 00:12:11,NaN,캐시워크,0,False,com.cashwalk.cashwalk
4,5,otfKOMf7VUaB5lxUARO0HNusxPB2,2018-12-03,2018-12-03 00:12:20,NaN,캐시워크,0,False,com.cashwalk.cashwalk


## Data Preprocessing

1. to Datetime
2. Drop Background(e.g.,backround app - cashwalk)
3. App Categorizing (with Crawling)
    - Result : [uid, date, stime, etime, packageName, Category]
4. Time Mapping : to count total duration of app usage
    - Result : [timestamp, package, Category]

### To Datetime

util.toDatetime()

 - df : dataframe 
 - timeCol : columns of timestamp (ex, [timestamp] or [stime, etime])
 - format : data format (format without any space, reg)
 - usage : toDatetime(df, ['stime', 'etime'], '%Y-%m-%d %H:%M:%S')



In [3]:
app_sample1 = util.toDatetime(app_sample1, ['date'], '%Y%m%d')
app_sample1 = util.toDatetime(app_sample1, ['stime', 'etime'], '%Y%m%d%H%M%S')

### Drop background


check if background or not
- total == 0
- isInUsable == False

In [4]:
app_sample1 = app_sample1[app_sample1['total'] != 0]
app_sample1 = app_sample1[app_sample1['isInUsable'] == False]
app_sample1.shape

(1271, 9)

### App Categorizing (with Crawling)

In [6]:
cat_list = pd.read_csv('App_category/Cat_group.csv') #grouped category list

# ex.
util.getGoogleCategory('com.no.color', cat_list)

'Entertainment & Music'

In [7]:
app_sample1['App_category'] = app_sample1['packageFullName'].apply(lambda x : util.getGoogleCategory(x, cat_list))
app_sample1.to_csv('result/A_categorized.csv')

In [8]:
pd.read_csv('result/A_categorized.csv').head(3)

,Unnamed: 0,_id,uid,date,stime,etime,packageName,total,isInUsable,packageFullName,category
0,2,3,otfKOMf7VUaB5lxUARO0HNusxPB2,2018-12-03,2018-12-03 00:11:59,2018-12-03 00:12:10,캐시워크,11,True,com.cashwalk.cashwalk,Health & Fitness
1,17,20,otfKOMf7VUaB5lxUARO0HNusxPB2,2018-12-03,2018-12-03 00:13:21,2018-12-03 00:13:24,카카오톡,3,True,com.kakao.talk,Communication
2,19,22,otfKOMf7VUaB5lxUARO0HNusxPB2,2018-12-03,2018-12-03 00:13:32,2018-12-03 00:13:33,카카오톡,1,True,com.kakao.talk,Communication


### Time Mapping
- mapping minutes

    - ex. Category1 stime : 10:00:00 , etime 10:05:00
    - 10:00, 10:01, 10:02, 10:03, 10:04, 10:05 ==> Category1
    
    util.appTimeMapping(app_sample1, columns):
    params : columns
    - columns[0] = timestamp
    - columns[1] = packageFullName
    - columns[2] = category

    - columns[3] = stime
    - columns[4] = etime
    - columns[5] = uid
    
    
- drop dupilicate
    - 10:00:01, 10:00:02 --> one row
    1. second to 0
    2. drop same timestamp


In [ ]:
colums = ['date', 'packageFullName', 'App_category', 'stime', 'etime', 'uid']
app_sample1_time = util.appTimeMapping(app_sample1, columns):

In [ ]:
app_sample_result = util.removeDuplicate(sample, timeColName, pkColName):


app_sample1_time['timestamp'] = pd.to_datetime(app_sample1_time['timestamp'], format='%Y-%m-%d %H:%M:%S')
app_sample1_time['timestamp'] = app_sample1_time['timestamp'].apply(lambda t: t.replace(second=0))

delDup = removeDuplicate(app_sample1_time, 'timestamp', 'packageFullName')
delDup.to_csv('result/A_timeMapped.csv')

In [9]:
pd.read_csv('result/A_timeMapped.csv').head(10)

,Unnamed: 0,Unnamed: 0.1,timestamp,packageName,category,uid
0,0,0,2018-12-03 00:14:00,카카오톡,Communication,otfKOMf7VUaB5lxUARO0HNusxPB2
1,1,1,2018-12-03 00:25:00,카카오톡,Communication,otfKOMf7VUaB5lxUARO0HNusxPB2
2,2,2,2018-12-03 00:34:00,Facebook,Communication,otfKOMf7VUaB5lxUARO0HNusxPB2
3,3,3,2018-12-03 00:35:00,Facebook,Communication,otfKOMf7VUaB5lxUARO0HNusxPB2
4,4,4,2018-12-03 00:36:00,Facebook,Communication,otfKOMf7VUaB5lxUARO0HNusxPB2
5,5,5,2018-12-03 00:37:00,Facebook,Communication,otfKOMf7VUaB5lxUARO0HNusxPB2
6,6,6,2018-12-03 00:38:00,Facebook,Communication,otfKOMf7VUaB5lxUARO0HNusxPB2
7,7,7,2018-12-03 00:39:00,Facebook,Communication,otfKOMf7VUaB5lxUARO0HNusxPB2
8,8,8,2018-12-03 00:40:00,Facebook,Communication,otfKOMf7VUaB5lxUARO0HNusxPB2
9,9,9,2018-12-03 00:41:00,Facebook,Communication,otfKOMf7VUaB5lxUARO0HNusxPB2
